In [1]:
import numpy as np
import GCRCatalogs
from astropy.table import Table

from clmm.structures import GalaxyCluster

In [2]:
# fyi this is clmm/structures/cluster.py

import pickle
import astropy.table
from astropy.table import Table

class GalaxyCluster():
    '''
    Object that contains the galaxy cluster metadata and background galaxy data
    Attributes
    ----------
    id (int): Unique identifier of the galaxy cluster
    ra (float): Right ascension of galaxy cluster center (in degrees)
    dec (float): Declination of galaxy cluster center (in degrees)
    z (float): Redshift of galaxy cluster center (in degrees)
    richness (int): Number of member galaxies above prescribes luminosity cut
    '''
    def __init__(self, id: int=None, 
                       ra: float=None, dec: float=None,
                       z: float=None, richness: int=None,
                       gals: astropy.table.table.Table=Table()
                ):
        self.id = id
        self.ra = ra
        self.dec = dec
        self.z = z
        self.richness = richness
        self.gals = gals

    def save(self, filename, **kwargs):
        """Saves GalaxyCluster object to filename using Pickle"""
        with open(filename, 'wb') as f:
            pickle.dump(self, f, **kwargs)

    def load(self, filename, **kwargs):
        """Loads GalaxyCluster object from filename using Pickle"""
        with open(filename, 'rb') as f:
            new_cl = pickle.load(f, **kwargs)
        self.__init__(**(new_cl.__dict__))


In [3]:
# fyi this is clmm/desc/load_from_dc2.py

import os
import numpy as np
import GCRCatalogs
from astropy.table import Table

from clmm.structures import GalaxyCluster

def load_from_dc2(N, catalog, save_dir):
    catalog = GCRCatalogs.load_catalog(catalog)
    halos = catalog.get_quantities(['galaxy_id', 'halo_mass', 'redshift','ra', 'dec'],
                                   filters=['halo_mass > 1e14','is_central==True'])

    for _ in range(N):
        i = np.random.randint(len(halos))

        id_cl = halos['galaxy_id'][i]
        ra_cl = halos['ra'][i]
        dec_cl = halos['dec'][i]
        z_cl = halos['redshift'][i]
        m_cl = halos['halo_mass'][i]

        # get galaxies around cluster
        ra_min, ra_max = ra_cl-0.3, ra_cl+0.3
        dec_min, dec_max = dec_cl-0.3, dec_cl+0.3
        z_min = z_cl + 0.1
        z_max = 1.5

        coord_filters = [
            'ra >= %s'%ra_min,
            'ra < %s'%ra_max,
            'dec >= %s'%dec_min,
            'dec < %s'%dec_max,
        ]
        z_filters = ['redshift >= %s'%z_min,
                     'redshift < %s'%z_max]

        gals = catalog.get_quantities(['galaxy_id', 'ra', 'dec', 'shear_1', 'shear_2',
                                       'redshift', 'convergence'], filters=(coord_filters + z_filters))

        # calculate reduced shear
        g1 = gals['shear_1']/(1.-gals['convergence'])
        g2 = gals['shear_2']/(1.-gals['convergence'])

        # store the results into an astropy table
        t = Table([gals['galaxy_id'],gals['ra'],gals['dec'],
                   2*g1, 2*g2,
                   gals['redshift'],gals['convergence']],
                  names=('id','ra','dec', 'e1', 'e2', 'z', 'kappa'))


        c = GalaxyCluster(id=id_cl, ra=ra_cl, dec=dec_cl,
                          z=z_cl, richness=None,
                          gals=t)

        c.save(os.path.join(save_dir, '%s.p'%id_cl))

In [4]:
catalog = 'cosmoDC2_v1.1.4_small'
save_dir = '/global/homes/m/mho1/scratch/clmm_scratch/cl_from_gcr'

load_from_dc2(5, catalog, save_dir)

In [5]:
# code to generate mock data TBD!

# https://github.com/LSSTDESC/CLMM/blob/issue/110/re-refactor-clmm-api/examples/generate_mock_data.ipynb

import clmm.mock_data as mock
from astropy.table import Table
#import clmm
import clmm.structures as structures


config={}
config['cluster_m'] = 1.e15
config['cluster_z'] = 0.3
config['src_z'] = 0.8
config['concentration'] = 4
config['cosmo'] = 'WMAP7-ML'
config['ngals'] = 10000
config['mdef'] = '200c'

ideal_data = mock.MockData(config=config)
noisy_data = mock.MockData(config=config)
# galaxies following redshift distribution, redshift error, shape noise
#noisy_data_z = mock.MockData(config=config)

ideal_data.generate()
noisy_data.generate(is_shapenoise=True, shapenoise=0.005)
#noisy_data_z.generate(is_shapenoise=True, is_zdistribution=True, is_zerr=True)

print(ideal_data.catalog[0])
print(noisy_data.catalog[0])

def save_cluster(cluster_id, mock_data):
    cluster = structures.GalaxyCluster()
    cluster.id = cluster_id
    cluster.ra = 0.
    cluster.dec = 0.
    cluster.z = 0.
    cluster.gals = mock_data.catalog
    cluster_filename = "%d.fits"%cluster_id
    cluster.save(cluster_filename)

save_cluster(123, ideal_data)
save_cluster(456, noisy_data)

# Test
cluster_test = structures.GalaxyCluster()
cluster_test.load("123.fits")
print(cluster_test)
print(cluster_test.gals)


 id       ra             dec              gamma1            gamma2       z 
--- -------------- ---------------- ----------------- ----------------- ---
  0 0.200718618275 -0.0967752041324 -0.00681586945514 -0.00856304220813 0.8
 id       ra            dec             gamma1           gamma2      z 
--- -------------- -------------- ----------------- --------------- ---
  0 0.189231740125 0.109807524399 -0.00790488968467 0.0138315465227 0.8
 id         ra              dec              gamma1            gamma2        z 
---- ---------------- ---------------- ----------------- ------------------ ---
   0   0.200718618275 -0.0967752041324 -0.00681586945514  -0.00856304220813 0.8
   1   0.132076256653   0.264715377373  0.00446830692164     0.005936668145 0.8
   2 -0.0955965981388  -0.110188250447  0.00264495006303     0.018556981282 0.8
   3   0.135850494568   0.217190831003    0.003967913652   0.00815386298934 0.8
   4  -0.265474773726  0.0327278640007 -0.00829055270294   -0.0020756732918 